In [1]:
# Setup display width, load packages, import symbols
ENV["COLUMNS"]=72
using Pkg; for p in ("Knet","Plots","PyCall","JSON","JLD2","FileIO","ImageMagick"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Base.Iterators: flatten
using Statistics: mean
using Plots;
using FileIO;
using MAT;
using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, Param,progress, sgd,dropout, relu,
Data,sigm, xavier,gpu,softmax
import Knet:param,param0
using Images;
using PyCall,JSON,JLD2,Random
@pyimport numpy as np 

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:400
└ @ PyCall /home/ege/.julia/packages/PyCall/RQjD7/src/PyCall.jl:400


In [2]:
atype()=(gpu() >= 0 ? KnetArray{Float32} : Array{Float32})
param(d...; init=xavier, atype=atype())=Param(atype(init(d...)))
param0(d...; atype=atype())=param(d...; init=zeros, atype=atype)

Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /buildworker/worker/package_linux64/build/src/locks.h:138 [inlined]
jl_typeinf_end at /buildworker/worker/package_linux64/build/src/gf.c:2568
typeinf_ext at ./compiler/typeinfer.jl:575
typeinf_ext at ./compiler/typeinfer.jl:611
jfptr_typeinf_ext_1.clone_1 at /home/ege/Downloads/julia-1.0.3-linux-x86_64/julia-1.0.3/lib/julia/sys.so (unknown line)
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2184
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1537 [inlined]
jl_apply_with_saved_exception_state at /buildworker/worker/package_linux64/build/src/rtutils.c:257
jl_type_infer at /buildworker/worker/package_linux64/build/src/gf.c:275
jl_compile_method_internal at /buildworker/worker/package_linux64/build/src/gf.c:1786 [inlined]
jl_fptr_trampoline at /buildworker/worker/package_linux64/build/src/gf.c:1830
jl_apply_generic at /buildworker/worker/package_linux

param0 (generic function with 1 method)

In [2]:
struct Chain
    layers
    Chain(layers...)=new(layers)
end
(c::Chain)(x) = (for layer in c.layers; x=layer(x) ;println(size(x));end; x) #layerların çıktılarını hesaplayıp son layerın çıktısını veriyor
(c::Chain)(x,y)=nll(c(x),y) #loss
(c::Chain)(d::Data)=mean(c(x,y) for (x,y) in d) #batch için mean loss hesapla

In [4]:
struct Fully; w;b;func;end;
Fully(m::Int,n::Int,func)=Fully(param(n,m),param0(n),func)
(ful::Fully)(x)=ful.func.(ful.w*x.+ful.b)

In [4]:
struct Conv; w;b;func;
    Conv(w,b)=new(Param(w),Param(b),relu)
    Conv(w,b,func)=new(Param(w),Param(b),func)
    end;
(c::Conv)(x)=c.func.(conv4(c.w,x,padding=1,mode=1).+c.b)
#(c::Conv)(x)=conv4(c.w,x,padding=1,mode=1).+c.b

In [5]:
struct Pool end
(p::Pool)(x)=pool(x)

In [68]:
struct Dense; w;b;func;
    Dense(w,b)=new(Param(w),Param(b),relu)
    Dense(w,b,func)=new(Param(w),Param(b),func)
    end;
function (den::Dense)(x) 
    matrix_multip=den.w*mat(x).+den.b
    if den.func==relu
        return den.func.(matrix_multip)
    else
        return den.func(matrix_multip)
    end
end

In [7]:
weight_dict=matread("imagenet-vgg-verydeep-16.mat")
layers=weight_dict["layers"]
w_arr=[]
for layer in layers
    if occursin("conv",layer["name"])
        w=layer["weights"]
        push!(w_arr,w[1])
        b=reshape(w[2], (1,1,length(w[2]),1))
        push!(w_arr,b)
        elseif occursin("fc",layer["name"])
            w=layer["weights"]
            w[1] = transpose(mat(w[1]))
            push!(w_arr,w[1])
            push!(w_arr,w[2])
        end
end

In [8]:
function sof(x)
    exp.(Knet.logp(x,dims=1))
end

sof (generic function with 1 method)

In [69]:
Vgg_model=Chain(Conv(w_arr[1],w_arr[2]),Conv(w_arr[3],w_arr[4]),Pool(),   #,  #layer1
    Conv(w_arr[5],w_arr[6]),Conv(w_arr[7],w_arr[8]),Pool(),              #layer2
    Conv(w_arr[9],w_arr[10]),Conv(w_arr[11],w_arr[12]),Conv(w_arr[13],w_arr[14]),Pool(),       #layer3
    Conv(w_arr[15],w_arr[16]),Conv(w_arr[17],w_arr[18]),Conv(w_arr[19],w_arr[20]),Pool(),       #layer4
    Conv(w_arr[21],w_arr[22]),Conv(w_arr[23],w_arr[24]),Conv(w_arr[25],w_arr[26]),Pool(),       #layer5
    Dense(w_arr[27],w_arr[28]),Dense(w_arr[29],w_arr[30]),Dense(w_arr[31],w_arr[32],softmax));#prediction burada

In [47]:
function preprocesscroppedimage(img::AbstractMatrix{<:AbstractRGB})
    z = (channelview(img).* 255)# .- 128)./128;
    _atype(permutedims(z, (2, 3, 1))[:,:,:,:])
    z=reverse(z,dims=3)
    #_atype(z[:,:,:,:])
end

preprocesscroppedimage (generic function with 1 method)

In [11]:
function image_data(img)
    global _imgcache
    if !@isdefined(_imgcache); _imgcache = Dict(); end
        a0=img
        new_size = ntuple(i->div(size(a0,i)*224,minimum(size(a0))),2)
        a1 = Images.imresize(a0, new_size)
        i1 = div(size(a1,1)-224,2)
        j1 = div(size(a1,2)-224,2)
        b1 = a1[i1+1:i1+224,j1+1:j1+224]
        # ad-hoc solution for Mac-OS image
        macfix = convert(Array{FixedPointNumbers.Normed{UInt8,8},3}, channelview(b1))
        c1 = permutedims(macfix, (3,2,1))
        d1 = convert(Array{Float32}, c1)
        e1 = reshape(d1[:,:,1:3], (224,224,3,1))
        averageImage=[123.68 116.779 103.939]
        f1 = (255 * e1)#[:,:,1,:] .- averageImage)
        f1[:,:,1,:].-=averageImage[1]
        f1[:,:,2,:].-=averageImage[2]
        f1[:,:,3,:].-=averageImage[3]
        g1 = permutedims(f1, [2,1,3,4])
    return g1
end


image_data (generic function with 1 method)

In [70]:
img=image_data(load("bee.jpg"));

In [71]:
pred=Vgg_model(img);

(224, 224, 64, 1)
(224, 224, 64, 1)
(112, 112, 64, 1)
(112, 112, 128, 1)
(112, 112, 128, 1)
(56, 56, 128, 1)
(56, 56, 256, 1)
(56, 56, 256, 1)
(56, 56, 256, 1)
(28, 28, 256, 1)
(28, 28, 512, 1)
(28, 28, 512, 1)
(28, 28, 512, 1)
(14, 14, 512, 1)
(14, 14, 512, 1)
(14, 14, 512, 1)
(14, 14, 512, 1)
(7, 7, 512, 1)
(4096, 1)
(4096, 1)
(1000, 1)


In [72]:
confidence,class_index=findmax(pred)
class_index=class_index[1]
println(confidence)
f = open("imagenet_classes.txt")
lines=readlines(f);
count=1
for i in lines
    if count==class_index
        println(i)
        break
    end
    count+=1
end
close(f)

0.9873051
bee
